In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from lbg_desc_forecast import load_joint_forecast, fig_dir

In [ ]:
def plot_relative_FoM(year: int, ax1: plt.Axes, ax2: plt.Axes):
    # Load both individual forecasts
    # Get list of parameters to fix
    p_to_fix = ["dz", "f_interlopers", "g_bias_inter", "mag_bias"]
    fix = []
    for p in p_to_fix:
        for band in "ugr":
            fix.append(f"{band}_{p}")
    srd, lbg, joint = load_joint_forecast(year=year, clean=True, fix=fix)

    # Function to print relative FoM in corner of plot
    def print_fom(ax: plt.Axes, params: list[str], y_offset: float = 0) -> None:
        # Calculate relative FOMs
        fom_srd = 1.0
        fom_lbg = lbg.figure_of_merit(params) / srd.figure_of_merit(params)
        fom_jnt = joint.figure_of_merit(params) / srd.figure_of_merit(params)

        # Print text
        y = np.array([0.25, 0.15, 0.05]) + y_offset
        settings = dict(transform=ax.transAxes, va="center", ha="left", fontsize=11)
        ax.text(0.03, y[0], f"{fom_srd:.2f}", c="k", **settings)
        ax.text(0.03, y[1], f"{fom_lbg:.2f}", c="C1", **settings)
        ax.text(0.03, y[2], f"{fom_jnt:.2f}", c="C0", **settings)

    # S8 vs Omega_M
    # -------------
    if ax1 is not None:
        ax1.set(
            xlabel="$\Omega_M$",
            ylabel="$S_8$",
        )

        omegaM, sigma8 = srd.fix(["w0", "wa"]).contour_2d(["Omega_M", "sigma8"])
        s8 = np.sqrt(omegaM / 0.3) * sigma8
        ax1.plot(omegaM, s8, c="k", ls="--")

        omegaM, sigma8 = lbg.fix(["w0", "wa"]).contour_2d(["Omega_M", "sigma8"])
        s8 = np.sqrt(omegaM / 0.3) * sigma8
        ax1.plot(omegaM, s8, c="C1")

        omegaM, sigma8 = joint.fix(["w0", "wa"]).contour_2d(["Omega_M", "sigma8"])
        s8 = np.sqrt(omegaM / 0.3) * sigma8
        ax1.plot(omegaM, s8, c="C0")

        print_fom(ax1, ["Omega_M", "sigma8"], 0.67)

    # w_a vs w_0
    # -------------
    if ax2 is not None:
        ax2.set(
            xlabel="$w_0$",
            ylabel="$w_a$",
        )

        w0, wa = srd.contour_2d(["w0", "wa"])
        ax2.plot(w0, wa, c="k", ls="--")

        w0, wa = lbg.contour_2d(["w0", "wa"])
        ax2.plot(w0, wa, c="C1")

        w0, wa = joint.contour_2d(["w0", "wa"])
        ax2.plot(w0, wa, c="C0")

        print_fom(ax2, ["w0", "wa"])

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(5, 5))

plot_relative_FoM(1, axes[0, 0], axes[1, 0])
plot_relative_FoM(10, axes[0, 1], axes[1, 1])

axes[0, 1].set_xlim(axes[0, 0].get_xlim())
axes[0, 1].set_ylim(axes[0, 0].get_ylim())
axes[0, 0].set_title("LSST year 1")
axes[0, 1].set_title("LSST year 10")

axes[1, 1].set_xlim(axes[1, 0].get_xlim())
axes[1, 1].set_ylim(axes[1, 0].get_ylim())
axes[1, 0].set_title("LSST year 1")
axes[1, 1].set_title("LSST year 10")


fig.subplots_adjust(hspace=0.5, wspace=0.5)

fig.savefig(fig_dir / "forecast_OmegaM_vs_S8_and_de.pdf", bbox_inches="tight")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(5, 2.2))

plot_relative_FoM(1, None, ax1)
plot_relative_FoM(10, None, ax2)

ax2.set_xlim(ax1.get_xlim())
ax2.set_ylim(ax1.get_ylim())

ax1.set_title("LSST year 1")
ax2.set_title("LSST year 10")

fig.subplots_adjust(wspace=0.4)
fig.savefig(fig_dir / "forecast_de_years1_and_10.pdf", bbox_inches="tight")